<a href="https://colab.research.google.com/github/mvdheram/Conference-Track-Management-/blob/main/Conference_track_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read inputs 

In [1]:
def readFromFile(file_name):
  file = open(file_name,'r')
  try:
    text = file.read()
    return text
  except IOError:
    return False
  finally:
    file.close()

In [2]:
txtfile = readFromFile('/content/Events_list.txt')

# Parse inputs into dict with talk_name and time

In [81]:
# Constants
MORNING_SESSION_TOTAL_MINUTES = 180
LUNCH_SESSION_TOTAL_MINUTES = 60
AFTERNOON_SESSION_TOTAL_MINUTES = 240 
NETWORKING_SESSION_TOTAL_MINUTES = 60 - AFTERNOON_SESSION_TOTAL_MINUTES
# >180 and <240 min
ENDS_WITH_LIGHTNING_REGEX = "lightning$"
ENDS_WITH_MINUTES_REGEX = "\d+min$"
NETWORKING_SESSION = 'Networking'
LUNCH_SESSION = 'Lunch'


MORNING_SESSION_START_TIME = 9
LUNCH_SESSION_START_TIME = 12
AFTERNOON_SESSION_START_TIME = 13
NETWORKING_SESSION_START_TIME = 17
LIGHTNING_MIN = 5

In [101]:
# Date_Time class
from datetime import datetime, date, time, timedelta

def get_time(hours):
  return time(hours)

def add_minutes_toTime(time,min):
  return (datetime.combine(date.today(),time)+ timedelta(minutes=int(min))).time()

def format_time(time):
  return time.strftime("%I:%M %p")

In [5]:
from typing_extensions import OrderedDict
import re

def parseTxtFile(textfile):
  talks= OrderedDict()
  lines = textfile.split('\n')
  for line in lines:
    try:
      if line.endswith('min'):
        talk_event = re.sub(ENDS_WITH_MINUTES_REGEX," ",line)
        talk_time = int(re.search(ENDS_WITH_MINUTES_REGEX,line).group(0).strip('min'))
      else:
        talk_event = re.sub(ENDS_WITH_LIGHTNING_REGEX," ",line)
        talk_time = 5
      talks[talk_event] = talk_time
    except AttributeError:
      pass
  return talks  

In [6]:
talksDict = parseTxtFile(txtfile)

# Solver

In [32]:
talksDict

OrderedDict([('Writing Fast Tests Against Enterprise Rails  ', 60),
             ('Overdoing it in Python  ', 45),
             ('Lua for the Masses  ', 30),
             ('Ruby Errors from Mismatched Gem Versions  ', 45),
             ('Common Ruby Errors  ', 45),
             ('Rails for Python Developers  ', 5),
             ('Communicating Over Distance  ', 60),
             ('Accounting-Driven Development  ', 45),
             ('Woah  ', 30),
             ('Sit Down and Write  ', 30),
             ('Pair Programming vs Noise  ', 45),
             ('Rails Magic  ', 60),
             ('Ruby on Rails: Why We Should Move On  ', 60),
             ('Clojure Ate Scala (on my project)  ', 45),
             ('Programming in the Boondocks of Seattle  ', 30),
             ('Ruby vs. Clojure for Back-End Development  ', 30),
             ('Ruby on Rails Legacy App Maintenance  ', 60),
             ('A World Without HackerNews  ', 30),
             ('User Interface CSS in Rails Apps  ', 30)])

In [33]:
talksDict.values()

odict_values([60, 45, 30, 45, 45, 5, 60, 45, 30, 30, 45, 60, 60, 45, 30, 30, 60, 30, 30])

In [34]:
talksDict.keys()

odict_keys(['Writing Fast Tests Against Enterprise Rails  ', 'Overdoing it in Python  ', 'Lua for the Masses  ', 'Ruby Errors from Mismatched Gem Versions  ', 'Common Ruby Errors  ', 'Rails for Python Developers  ', 'Communicating Over Distance  ', 'Accounting-Driven Development  ', 'Woah  ', 'Sit Down and Write  ', 'Pair Programming vs Noise  ', 'Rails Magic  ', 'Ruby on Rails: Why We Should Move On  ', 'Clojure Ate Scala (on my project)  ', 'Programming in the Boondocks of Seattle  ', 'Ruby vs. Clojure for Back-End Development  ', 'Ruby on Rails Legacy App Maintenance  ', 'A World Without HackerNews  ', 'User Interface CSS in Rails Apps  '])

In [35]:
talks_set = set(talksDict.keys())

In [36]:
def talkTimeSum(tp):
  return sum([talksDict[key] for key in tp])

In [47]:
from itertools import cycle

def subsetSum(sessionTotalMinutes, talks):
  cumsum = 0
  session = []
  target  = sessionTotalMinutes
  temp_keys = []

  iterator = cycle(talks_set)
  session.clear()

  for talk in iterator:
    cumsum = talkTimeSum(temp_keys)

    if cumsum < target:
      temp_keys.append(talk)

    elif cumsum == target:
      return set(temp_keys)

    elif cumsum > target:
      temp_keys.pop(0)

  return session

In [48]:
def updateTalksList(sessionTalks):
  global talks_set 
  if len(talks_set) is not None:
     talks_set = talks_set.difference(sessionTalks)
     return talks_set
  else:
    return 0

In [49]:
morningSession = subsetSum(MORNING_SESSION_TOTAL_MINUTES,talks_set)

In [50]:
morningSession

{'Clojure Ate Scala (on my project)  ',
 'Lua for the Masses  ',
 'Pair Programming vs Noise  ',
 'Writing Fast Tests Against Enterprise Rails  '}

In [51]:
eveningSession = subsetSum(AFTERNOON_SESSION_TOTAL_MINUTES,updateTalksList(morningSession))

In [52]:
eveningSession2 = subsetSum(185,updateTalksList(eveningSession))

In [53]:
eveningSession

{'Common Ruby Errors  ',
 'Communicating Over Distance  ',
 'Overdoing it in Python  ',
 'Ruby on Rails Legacy App Maintenance  ',
 'Sit Down and Write  '}

In [54]:
eveningSession2

{'Accounting-Driven Development  ',
 'Rails for Python Developers  ',
 'Ruby Errors from Mismatched Gem Versions  ',
 'Ruby on Rails: Why We Should Move On  ',
 'Woah  '}

In [55]:
morningSession2 = subsetSum(180,updateTalksList(eveningSession2))

In [56]:
morningSession2

{'A World Without HackerNews  ',
 'Programming in the Boondocks of Seattle  ',
 'Rails Magic  ',
 'Ruby vs. Clojure for Back-End Development  ',
 'User Interface CSS in Rails Apps  '}

# Outputting 

In [113]:
import numpy as np

def getCumSum(talkList):
  return list(np.cumsum(talkList))

In [153]:
def assignTimeTotalks(talkList, morning:bool):
  talktime = {}

  cumSum = getCumSum([talksDict[talk] for talk in talkList])
  first_item = list(talkList)[0]

  if morning == 1:
     # Add first element to dict for calculating the cumulative sum 
    start_time = MORNING_SESSION_START_TIME
    ends_with = LUNCH_SESSION
    ends_with_start_time = LUNCH_SESSION_START_TIME
    
  else:
    start_time = AFTERNOON_SESSION_START_TIME
    ends_with = NETWORKING_SESSION
    ends_with_start_time = NETWORKING_SESSION_START_TIME

  talktime[format_time(get_time(start_time))] = first_item, talksDict[first_item]

  for talk,cumsum in zip(list(talkList)[1:],cumSum) :
    talktime[format_time(add_minutes_toTime(get_time(start_time),cumsum))] = talk, talksDict[talk]
  talktime[format_time(get_time(ends_with_start_time))] = ends_with

  return talktime

In [155]:
morning =  subsetSum(MORNING_SESSION_TOTAL_MINUTES,talks_set)

In [156]:
assignTimeTotalks(morning, 1)

{'09:00 AM': ('Programming in the Boondocks of Seattle  ', 30),
 '09:30 AM': ('Rails Magic  ', 60),
 '10:30 AM': ('A World Without HackerNews  ', 30),
 '11:00 AM': ('Ruby vs. Clojure for Back-End Development  ', 30),
 '11:30 AM': ('User Interface CSS in Rails Apps  ', 30),
 '12:00 PM': 'Lunch'}

In [ ]:
import collections
tracks = collections.defaultdict(dict)

while talkTimeSum(talks_set != 0):
  id = 1
  tracks[id] = {}
  for talks in subsetSum(MORNING_SESSION_TOTAL_MINUTES,talks_set):
    tracks[id][get_time(MORNING_SESSION_START_TIME)] = talks, talksDict[talks]
    tracks[id][add_minutes_toTime(talksDict[talks])] = 